In [1]:
import numpy as np
import scipy.optimize as so
import itertools
from scipy.linalg import det
np.set_printoptions(threshold=np.inf)

In [9]:
#A max zadatak sa vezbi (revidirani simpleks)
C = np.array([6, 14, 13, 0, 0]) 

A1 = np.matrix([[.5, 2, 1, 1, 0],
     [1, 2, 4, 0, 1]])

b = np.array([24, 60])

p_type = 'max'
# C*-1 za maksimum
print('linprog :',so.linprog(C*-1, A_ub=A1, b_ub=b, options=dict(maxiter=100, bland=True),method="simplex").x)

linprog : [36.  0.  6.  0.  0.]


In [11]:
#B max zadatak sa vezbi (revidirani simpleks)
C = np.array([2, 1.5, 0, 0])

A1 = np.matrix([[6, 3, 1, 0],
     [75, 100, 0, 1]])

b = np.array([1200, 25000])

p_type = 'max'
print('linprog :',so.linprog(C*-1, A_ub=A1, b_ub=b, options=dict(maxiter=100, bland=True),method="simplex").x)

linprog : [120. 160.   0.   0.]


In [13]:
#C max zadatak sa vezbi, nije moguce uzeti dodatne kao bazne (dualni problem) 
C = np.array([24, 60, 0, 0, 0])

A1 = np.matrix([[-.5, -1, 1, 0, 0],
              [-2, -2, 0, 1, 0],
               [-1, -4, 0, 0, 1]])

b = np.array([-6, -14, -13])

p_type = 'min'
print('linprog :',so.linprog(C, A_ub=A1, b_ub=b,method="simplex").x)

linprog : [11.   0.5  0.   9.   0. ]


In [15]:
#D min zadatak sa predavanja

#C = np.array([-1, -2, 1, -3, -4, 2, 0, 0, 0])

C = np.array([-1, -2, 1, -1, -4, 2, 0, 0, 0])

A1 = np.matrix([[1, 1, 1, 1, 1, 1, 1, 0, 0],
               [2, -1, -2, 1, 0, 0, 0, 1, 0],
              [0, 0, 1, 1, 2, 1, 0, 0, 1]])

b = np.array([6, 4, 4])

p_type = 'min'

In [6]:
def count_zeroes(vector):
    num_zeroes = 0
    for cell in vector:
        if cell == 0:
            num_zeroes += 1
    return num_zeroes

# racunanje nove bazne
def get_base_vars(C, А1, b):
    # treba i druga inverzna matrica da se ubaci u tabelu
    bad_b_ = False
    num_base_vars = count_zeroes(C)
    base_vars = []

    
    for i in range(len(C)-num_base_vars, len(C)):
        base_vars.append(i+1)
# -----------------------------------------
    
    final_B = np.eye(num_base_vars)
    
    b_ = np.dot(np.linalg.inv(final_B),np.transpose(b))
    
    for i in b_:
        if i < 0:
            bad_b_ = True
    if bad_b_ == True:
        possible_base_vars = list(itertools.combinations(range(1, len(C)), num_base_vars))
        for comb in possible_base_vars:
            B = np.eye(num_base_vars)
            j = 0
            for i in comb:
                B[:,[j]] = A1[:,i-1]
                j+=1

            b_ = np.dot(np.linalg.inv(B),np.transpose(b))

            bad_b_ = False

            for i in b_:
                if i < 0:
                    bad_b_ = True

            if bad_b_ == False : 
                final_B = B
                base_vars = comb

    return base_vars, final_B

def calculate_pivot(matrix, C, A1):
    
    min_neg = 0
    max_pos = 0
    Z_value = 0
    w = matrix[0,1:len(matrix[1:,:])+1]
    pivot_index = 0
    end = False
    num_base_vars = len(matrix[1:,0])
    
    if p_type == 'min':
        for i in range(0,len(C)):
            if i+1 not in matrix[1:,0]:
                # i + 1 je broj varijable
                aj = A1[:,i]
                temp = np.dot(w, aj) - C[i]
                if max_pos < temp:
                    max_pos = temp
                    pivot_index = i+1
    
    else: 
        for i in range(0,len(C)):
            if i+1 not in matrix[1:,0]:
                aj = A1[:,i]
                temp = np.dot(w, aj) - C[i]
                if min_neg > temp:
                    min_neg = temp
                    pivot_index = i+1

    if p_type == 'min':
        if max_pos == 0:
            end = True
        Z_value = max_pos
    else:
        if min_neg == 0:
            end = True
        Z_value = min_neg
                    
    print('Z_value',p_type, ':', Z_value)
    
    B_inv = matrix[1:,1:num_base_vars+1]
    a = A1[:,pivot_index-1]
    y = np.dot(B_inv,a)
    
    #pivot_index isto sto i j u algoritmu u svesci
    return Z_value, pivot_index, np.squeeze(np.asarray(y)), end

# racuna sa koja bazna postaje slobodna
def get_new_base(matrix):
    num_base_vars = len(matrix[1:,0])
    pivot_column = matrix[1:,num_base_vars+2]
    b_1 = matrix[1:,num_base_vars+1]
    inf_end = False
    
    min_positive = b_1[0]/pivot_column[0]
    min_positive_index = matrix[1,0]
    
    for i in range(1,len(pivot_column)):

        temp = b_1[i]/pivot_column[i]

        
        if temp < min_positive and temp > 0:
            min_positive = temp
            min_positive_index = matrix[i+1,0]


    return min_positive_index, inf_end
    
def get_new_matrix(matrix, pivot_index, pivot_row, num_base_vars):
    
    new_sim_matrix = np.zeros((num_base_vars+1, num_base_vars+3))
    pivot_row_position = 0
    
    #bazne postavljanje
    for i in range(1,num_base_vars+1):
        
        if matrix[i,0] == pivot_row:
            new_sim_matrix[i,0] = pivot_index
            pivot_row_position = i
        else:
            new_sim_matrix[i,0] = matrix[i,0]
    e = matrix[pivot_row_position, num_base_vars+2]
    
    # ev, ek
    for row in range(num_base_vars+1):
        
        for col in range(1, num_base_vars+3):

            if row == pivot_row_position and col != num_base_vars+2:
                new_sim_matrix[row, col] = matrix[row, col]/e
            #ostali
            else:
                new_sim_matrix[row, col] = matrix[row, col] - (matrix[pivot_row_position, col]*matrix[row, num_base_vars+2])/e
    return new_sim_matrix

def get_Cb(C, num_base_vars):
    print('num_base_vars')
    print(num_base_vars)
    Cb = np.zeros(len(num_base_vars))
    j = 0
    for var in base_vars:
        Cb[j] = C[var-1]
        j+=1
    return Cb
        

In [7]:
def r_simplex(C, A1, b, p_type):
    
    num_base_vars = len(b) 
    base_vars, B = get_base_vars(C, A1, b)
    print(base_vars)
    B_inv = np.linalg.inv(B)
    Cb = get_Cb(C, base_vars)
    CbB_inv = [0]
    finish = False
    w = np.dot(Cb,B_inv)
    simplex_matrix = np.zeros((num_base_vars+1, num_base_vars+3))

    b_ = np.dot(B_inv,np.transpose(b))

    i = 1
    for var in base_vars:
        simplex_matrix[i][0] = var
        i = i+1
    simplex_matrix[1:num_base_vars+1,1:num_base_vars+1] = B_inv
    simplex_matrix[1:,num_base_vars+1] = b_
    simplex_matrix[0,1:num_base_vars+1] = w
    simplex_matrix[0,num_base_vars+1] = np.dot(np.transpose(Cb),b_)

    result = []
    while(not finish):
        print(simplex_matrix)
        result = simplex_matrix.copy()
        simplex_matrix[0,-1], pivot_index, simplex_matrix[1:,-1], finish = calculate_pivot(simplex_matrix, C, A1)
        new_base, inf_end  = get_new_base(simplex_matrix)
        simplex_matrix = get_new_matrix(simplex_matrix, pivot_index, new_base, num_base_vars)
        if inf_end: 
            print('Resenje neograniceno')
            break

    print('---------------------------------------------------------')
    print('Z', p_type,'=', result[0,num_base_vars+1])
    j = 1
    x = 'x'
    for i in range(num_base_vars):
        print(f'{x}{result[i+1,0]}', '=', result[j, num_base_vars+1])
        j = j+1
    print('Ostale su 0.')

In [16]:

num_base_vars = len(b) 
base_vars, B = get_base_vars(C, A1, b)
print(base_vars)
B_inv = np.linalg.inv(B)
Cb = get_Cb(C, base_vars)
CbB_inv = [0]
finish = False
w = np.dot(Cb,B_inv)
simplex_matrix = np.zeros((num_base_vars+1, num_base_vars+3))

b_ = np.dot(B_inv,np.transpose(b))

i = 1
for var in base_vars:
    simplex_matrix[i][0] = var
    i = i+1
simplex_matrix[1:num_base_vars+1,1:num_base_vars+1] = B_inv
simplex_matrix[1:,num_base_vars+1] = b_
simplex_matrix[0,1:num_base_vars+1] = w
simplex_matrix[0,num_base_vars+1] = np.dot(np.transpose(Cb),b_)

result = []
while(not finish):
    print(simplex_matrix)
    result = simplex_matrix.copy()
    simplex_matrix[0,-1], pivot_index, simplex_matrix[1:,-1], finish = calculate_pivot(simplex_matrix, C, A1)
    new_base, inf_end  = get_new_base(simplex_matrix)
    simplex_matrix = get_new_matrix(simplex_matrix, pivot_index, new_base, num_base_vars)
    if inf_end: 
        print('Resenje neograniceno')
        break

print('---------------------------------------------------------')
print('Z', p_type,'=', result[0,num_base_vars+1])
j = 1
x = 'x'
for i in range(num_base_vars):
    print(f'{x}{result[i+1,0]}', '=', result[j, num_base_vars+1])
    j = j+1
print('Ostale su 0.')

[7, 8, 9]
num_base_vars
[7, 8, 9]
[[0. 0. 0. 0. 0. 0.]
 [7. 1. 0. 0. 6. 0.]
 [8. 0. 1. 0. 4. 0.]
 [9. 0. 0. 1. 4. 0.]]
Z_value min : [[4.]]
[[ 0.   0.   0.  -2.  -8.   0. ]
 [ 7.   1.   0.  -0.5  4.   0. ]
 [ 8.   0.   1.   0.   4.   0. ]
 [ 5.   0.   0.   0.5  2.   0. ]]
Z_value min : [[2.]]
[[  0.   -2.    0.   -1.  -16.    0. ]
 [  2.    1.    0.   -0.5   4.    0. ]
 [  8.    1.    1.   -0.5   8.    0. ]
 [  5.    0.    0.    0.5   2.    0. ]]
Z_value min : 0
---------------------------------------------------------
Z min = -16.0
x2.0 = 4.0
x8.0 = 8.0
x5.0 = 2.0
Ostale su 0.


/tmp/ipykernel_31123/1065353479.py:109: RuntimeWarning: divide by zero encountered in double_scalars
  temp = b_1[i]/pivot_column[i]
